In [1]:
import tensorflow as tf

from tensorflow.keras.layers import LSTM

# from tensorflow.keras.models import Sequential

from tensorflow.keras.losses import MeanSquaredError
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
true_path = "C:\\Users\\rferr\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design\\CSV_Files\\0001_1404.038_stbd_FinalCleaned_RollReady.csv"
#training_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\\train1_port_new.csv"
#test_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\StbdTesting_1404_002.csv"
#test_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\\test1_stbd_new.csv"
prediction_path = "C:\\Users\\rferr\\Desktop\CSVs\\PredictedOutputTest_NEW_DATA_2.csv"

In [3]:
df_true=pd.read_csv(true_path)
#df_test=pd.read_csv(test_path)
df_prediction=pd.read_csv(prediction_path)


In [4]:

#df_predictedDOA = df_prediction.iloc[:,4]
#df_prediction
df_predictedDOA = df_prediction[["PredictedDOA"]]
df_predictedDOA

,PredictedDOA
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
...,...
3382107,74.402985
3382108,74.135681
3382109,74.490456
3382110,74.241730


In [5]:
df_reference = df_true[["DOA", "AngleUncertainty"]]
df_reference

,DOA,AngleUncertainty
0,3.690000,58
1,2.394000,19
2,1.950000,34
3,-0.588000,49
4,0.762000,25
...,...,...
3382107,76.692001,24
3382108,76.596001,16
3382109,76.350001,15
3382110,76.326001,21


In [6]:
df_reference['AngleUncertainty'].max()

105

In [7]:
df_reference['AngleUncertainty'].mean()

27.904559636109035

In [8]:
df_reference['AngleUncertainty'].var()

222.02346369026324

In [9]:
df_comparison = df_predictedDOA.join(df_reference)
df_comparison

,PredictedDOA,DOA,AngleUncertainty
0,0.000000,3.690000,58
1,0.000000,2.394000,19
2,0.000000,1.950000,34
3,0.000000,-0.588000,49
4,0.000000,0.762000,25
...,...,...,...
3382107,74.402985,76.692001,24
3382108,74.135681,76.596001,16
3382109,74.490456,76.350001,15
3382110,74.241730,76.326001,21


In [10]:
theShape = df_comparison.shape
numRows= theShape[0]

In [11]:
numCols = 1
#5504457 
df_results = pd.DataFrame(index=range(numRows),columns=range(numCols))
#df_results = pd.DataFrame({'PassFail'})
#df_comparison.insert()
df_results.columns = {"PassFail"}
df_results

,PassFail
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
3382107,NaN
3382108,NaN
3382109,NaN
3382110,NaN


In [12]:
df_comparison = df_comparison.join(df_results)
df_comparison

,PredictedDOA,DOA,AngleUncertainty,PassFail
0,0.000000,3.690000,58,NaN
1,0.000000,2.394000,19,NaN
2,0.000000,1.950000,34,NaN
3,0.000000,-0.588000,49,NaN
4,0.000000,0.762000,25,NaN
...,...,...,...,...
3382107,74.402985,76.692001,24,NaN
3382108,74.135681,76.596001,16,NaN
3382109,74.490456,76.350001,15,NaN
3382110,74.241730,76.326001,21,NaN


In [13]:
df_comparison['Delta'] = df_comparison['PredictedDOA'] - df_comparison['DOA']
df_comparison['AbsDelta'] = df_comparison['Delta'].abs()
df_comparison

,PredictedDOA,DOA,AngleUncertainty,PassFail,Delta,AbsDelta
0,0.000000,3.690000,58,NaN,-3.690000,3.690000
1,0.000000,2.394000,19,NaN,-2.394000,2.394000
2,0.000000,1.950000,34,NaN,-1.950000,1.950000
3,0.000000,-0.588000,49,NaN,0.588000,0.588000
4,0.000000,0.762000,25,NaN,-0.762000,0.762000
...,...,...,...,...,...,...
3382107,74.402985,76.692001,24,NaN,-2.289016,2.289016
3382108,74.135681,76.596001,16,NaN,-2.460320,2.460320
3382109,74.490456,76.350001,15,NaN,-1.859545,1.859545
3382110,74.241730,76.326001,21,NaN,-2.084271,2.084271


In [14]:
# less than (0.02*AngleUncertainty)/2, since this is the 2sigma value, and we want all values to be under 1 sigma.
df_comparison['PassFail'] = df_comparison['AbsDelta'] <= (0.02*df_comparison['AngleUncertainty']/2)
df_comparison

,PredictedDOA,DOA,AngleUncertainty,PassFail,Delta,AbsDelta
0,0.000000,3.690000,58,False,-3.690000,3.690000
1,0.000000,2.394000,19,False,-2.394000,2.394000
2,0.000000,1.950000,34,False,-1.950000,1.950000
3,0.000000,-0.588000,49,False,0.588000,0.588000
4,0.000000,0.762000,25,False,-0.762000,0.762000
...,...,...,...,...,...,...
3382107,74.402985,76.692001,24,False,-2.289016,2.289016
3382108,74.135681,76.596001,16,False,-2.460320,2.460320
3382109,74.490456,76.350001,15,False,-1.859545,1.859545
3382110,74.241730,76.326001,21,False,-2.084271,2.084271


In [15]:
PassCount = df_comparison['PassFail'].sum()
PassCount

342671

In [16]:
FailCount = (~df_comparison['PassFail']).sum()
FailCount

3039441

In [17]:
TotalCount = PassCount+FailCount
TotalCount

3382112

In [18]:
PercentPass = 100*(PassCount/TotalCount)
PercentPass

10.131864349850035